In [10]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, Flatten, MaxPooling2D, Input, BatchNormalization, Dropout, Dense

In [11]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [12]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
train_path = 'train'
validation_path = 'test'

batch_size = 64

train_datagen = ImageDataGenerator(
    rescale = 1 / 255.0,
    rotation_range = 10,
    zoom_range = 0.1,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1 / 255.0
)

train_generator = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (48,48),
    color_mode = 'grayscale',
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True,
    seed = 42
)

valid_generator = test_datagen.flow_from_directory(
    directory = validation_path,
    target_size = (48,48),
    color_mode = 'grayscale',
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True,
    seed = 42
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [14]:
# model definition

input = Input(shape = (48,48,1))
x = Conv2D(filters=256,kernel_size=3,activation='relu',padding='same')(input)

x = Conv2D(filters=512,kernel_size=3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)

x = Conv2D(filters=384,kernel_size=3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)

x = Conv2D(filters=192,kernel_size=3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)

x = Conv2D(filters=384,kernel_size=3,activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.4)(x)

x = Flatten()(x)

x = Dense(256,activation='relu')(x)
x = BatchNormalization()(x)

x = Dropout(0.3)(x)
x = Dense(7,activation='softmax')(x)

model = Model(input, x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 48, 48, 256)       2560      
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 512)       1180160   
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 512)      2048      
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 512)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 512)       0     

In [17]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience = 4, verbose = 1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience = 10, verbose = 1)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss = tf.keras.losses.categorical_crossentropy, metrics = 'acc', optimizer = optimizer)
history = model.fit(train_generator,validation_data = valid_generator, epochs = 100, callbacks = [early_stopping])

Epoch 1/100
449/449 [==============================] - 219s 466ms/step - loss: 2.0030 - acc: 0.2479 - val_loss: 1.8917 - val_acc: 0.2665
Epoch 2/100
449/449 [==============================] - 206s 458ms/step - loss: 1.6531 - acc: 0.3561 - val_loss: 1.5113 - val_acc: 0.4171
Epoch 3/100
449/449 [==============================] - 206s 458ms/step - loss: 1.4456 - acc: 0.4423 - val_loss: 1.6602 - val_acc: 0.3553
Epoch 4/100
449/449 [==============================] - 205s 456ms/step - loss: 1.3214 - acc: 0.4942 - val_loss: 1.2361 - val_acc: 0.5217
Epoch 5/100
449/449 [==============================] - 205s 457ms/step - loss: 1.2571 - acc: 0.5234 - val_loss: 1.2066 - val_acc: 0.5344
Epoch 6/100
449/449 [==============================] - 205s 456ms/step - loss: 1.2146 - acc: 0.5413 - val_loss: 1.2441 - val_acc: 0.5269
Epoch 7/100
449/449 [==============================] - 206s 457ms/step - loss: 1.1827 - acc: 0.5507 - val_loss: 1.1082 - val_acc: 0.5755
Epoch 8/100
449/449 [====================

In [18]:
model.evaluate(valid_generator,verbose=1)

113/113 [==============================] - 15s 127ms/step - loss: 0.9432 - acc: 0.6606


[0.9431591033935547, 0.6606296896934509]

In [21]:
model.save('models/55epES.h5');